**Base de dados de Games**

*   Prever vendas de Games
*   Regressão com múltiplas saídas



**Importações Iniciais**




In [0]:
import pandas as pd
from keras.layers import Dense, Dropout, Activation, Input
from keras.models import Model

**Leitura da base de dados e criação do DataFrame**

In [0]:
df = pd.read_csv('games.csv')

**Visualização e análise dos dados do DataFrame**

In [0]:
df

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


A função ***describe*** éé utilizada para mostrar dados estatísticos do DataFrame

*   Value counts
*   Mean
*   std(desvio padrão)
*   porcentagens
*   valores máximos
*   valores minímos









In [0]:
df.describe()

,Year_of_Release,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Count
count,16450.000000,16719.000000,16719.000000,16719.000000,16719.000000,16719.000000,8137.000000,8137.000000,7590.000000
mean,2006.487356,0.263330,0.145025,0.077602,0.047332,0.533543,68.967679,26.360821,162.229908
std,5.878995,0.813514,0.503283,0.308818,0.186710,1.547935,13.938165,18.980495,561.282326
min,1980.000000,0.000000,0.000000,0.000000,0.000000,0.010000,13.000000,3.000000,4.000000
25%,2003.000000,0.000000,0.000000,0.000000,0.000000,0.060000,60.000000,12.000000,10.000000
50%,2007.000000,0.080000,0.020000,0.000000,0.010000,0.170000,71.000000,21.000000,24.000000
75%,2010.000000,0.240000,0.110000,0.040000,0.030000,0.470000,79.000000,36.000000,81.000000
max,2020.000000,41.360000,28.960000,10.220000,10.570000,82.530000,98.000000,113.000000,10665.000000


**Apagando algumas colunas que não influenciam na previsão das targets**




In [0]:
df = df.drop('Other_Sales', axis = 1)
df = df.drop('Developer', axis = 1)
df = df.drop('Name', axis = 1)
df = df.drop('NA_Sales', axis = 1)
df = df.drop('EU_Sales', axis = 1)
df = df.drop('JP_Sales', axis = 1)


**Selecionando colunas do tipo objeto**

*   Com as colunas do tipo objeto(variáveis categóricas), podemos realizar o label encoder. Ou seja, transformar essas variáveis para tipos númericos. 

*   Com a aplicação da função Label Encoder, podemos substituir os valores nulos pela média. Assim, não perderíamos uma grande quantidade de dados, caso optássemos por apagar os dados do tipo NaN.





In [0]:
df_objeto = df.select_dtypes(include=[object]).columns

In [0]:
df_objeto

Index(['Platform', 'Genre', 'Publisher', 'User_Score', 'Rating'], dtype='object')

**Importação da função Label Encoder**

*   Importação da função e aplicação da mesma
*   Laço para percorrer a lista de objetos(colunas categóricas) e transformando-as em variáveis númericas



In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder = LabelEncoder()

In [0]:
for coluna in df_objeto:
  if coluna in df:
    df[coluna] = label_encoder.fit_transform(df[coluna].astype(str))

Visualização de como o df está, após a transformação

In [0]:
df

,Platform,Year_of_Release,Genre,Publisher,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,26,2006.0,10,361,82.53,76.0,51.0,77,322.0,1
1,11,1985.0,4,361,40.24,NaN,NaN,95,NaN,8
2,26,2008.0,6,361,35.52,82.0,73.0,80,709.0,1
3,26,2009.0,10,361,32.77,80.0,73.0,77,192.0,1
4,5,1996.0,7,361,31.37,NaN,NaN,95,NaN,8
...,...,...,...,...,...,...,...,...,...,...
16714,17,2016.0,0,503,0.01,NaN,NaN,95,NaN,8
16715,28,2006.0,10,91,0.01,NaN,NaN,95,NaN,8
16716,20,2016.0,1,233,0.01,NaN,NaN,95,NaN,8
16717,6,2003.0,4,549,0.01,NaN,NaN,95,NaN,8


Visualizando a quantidade de valores NaN que temos no DataFrame

In [0]:
df.isnull().sum()

Platform              0
Year_of_Release     269
Genre                 0
Publisher             0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score            0
User_Count         9129
Rating                0
dtype: int64

**Pré-processamento dos dados**


*   Tratamento de dados faltantes

*   Divisão do DataFrame em:
    1. Features
    2. venda_na
    3. venda_eu
    4. venda_jp





**Substituindo valores NaN pela média da coluna**


In [0]:
df.isnull().sum()

Platform              0
Year_of_Release     269
Genre                 0
Publisher             0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score            0
User_Count         9129
Rating                0
dtype: int64

In [0]:
df.update(df['Year_of_Release'].fillna(df['Year_of_Release'].mean()))
df.update(df['Critic_Score'].fillna(df['Critic_Score'].mean()))
df.update(df['Critic_Count'].fillna(df['Critic_Count'].mean()))
df.update(df['User_Count'].fillna(df['User_Count'].mean()))

In [0]:
df.isnull().sum()

Platform           0
Year_of_Release    0
Genre              0
Publisher          0
Global_Sales       0
Critic_Score       0
Critic_Count       0
User_Score         0
User_Count         0
Rating             0
dtype: int64

**Divisão do DataFrame**

*   features
*   venda_na
*   venda_eu
*   venda_jp





In [0]:
df

,Platform,Year_of_Release,Genre,Publisher,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,26,2006.0,10,361,82.53,76.000000,51.000000,77,322.000000,1
1,11,1985.0,4,361,40.24,68.967679,26.360821,95,162.229908,8
2,26,2008.0,6,361,35.52,82.000000,73.000000,80,709.000000,1
3,26,2009.0,10,361,32.77,80.000000,73.000000,77,192.000000,1
4,5,1996.0,7,361,31.37,68.967679,26.360821,95,162.229908,8
...,...,...,...,...,...,...,...,...,...,...
16714,17,2016.0,0,503,0.01,68.967679,26.360821,95,162.229908,8
16715,28,2006.0,10,91,0.01,68.967679,26.360821,95,162.229908,8
16716,20,2016.0,1,233,0.01,68.967679,26.360821,95,162.229908,8
16717,6,2003.0,4,549,0.01,68.967679,26.360821,95,162.229908,8


In [0]:
features = df.iloc[:, [0,1,2,3,5,6,7,8,9]].values
alvo = df.iloc[:,4].values

**Utilização da função OneHotEncoder, para atributos categóricos que são representados por sequências**

In [0]:
from sklearn.compose import ColumnTransformer
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0,2,3,8])],remainder='passthrough') 
features = onehotencoder.fit_transform(features).toarray()

**Estrutura da Rede Neural**


*   Regressão de múltiplas saídas



In [0]:
def criarRede():
    camada_entradas = Input(shape=(640,))
    ativacao = Activation(activation = 'sigmoid')
    camada_oculta1 = Dense(units = 32, activation=ativacao)(camada_entradas)
    camada_oculta2 = Dense(units=2, activation=ativacao)(camada_oculta1)
    camada_saida = Dense(units=1, activation='linear')(camada_oculta2)
    regressor = Model(inputs = camada_entradas,
                      outputs = [camada_saida])
      
    regressor.compile(optimizer='adam', loss='mse')
    return regressor

In [0]:
regressor = KerasRegressor(build_fn=criarRede,
                         epochs = 50,
                         batch_size = 10)

**Validação Cruzada**

In [0]:
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasRegressor

In [0]:
resultados = cross_val_score(estimator=regressor, X = features, y = alvo,
                             cv = 10, scoring = 'neg_mean_absolute_error')

/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as Activation) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/50
15047/15047 [==============================] - 2s 122us/step - loss: 0.0778
Epoch 2/50
15047/15047 [==============================] - 2s 118us/step - loss: 0.0693
Epoch 3/50
15047/15047 [==============================] - 2s 125us/step - loss: 0.0694
Epoch 4/50
15047/15047 [==============================] - 2s 122us/step - loss: 0.0693
Epoch 5/50
15047/15047 [==============================] - 2s 130us/step - loss: 0.0693
Epoch 6/50
15047/15047 [==============================] - 2s 123us/step - loss: 0.0697
Epoch 7/50
15047/15047 [==============================] - 2s 124us/step - loss: 0.0695
Epoch 8/50
15047/15047 [==============================] - 2s 115us/step - loss: 0.0696
Epoch 9/50
15047/15047 [==============================] - 2s 115us/step - loss: 0.0696
Epoch 10/50
15047/15047 [==============================] - 2s 116us/step - loss: 0.0696
Epoch 11/50
15047/15047 [==============================] - 2s 121us/step - loss: 0.0696
Epoch 12/50
15047/15047 [================

/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as Activation) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/50
15047/15047 [==============================] - 2s 123us/step - loss: 2.6905
Epoch 2/50
15047/15047 [==============================] - 2s 113us/step - loss: 2.6471
Epoch 3/50
15047/15047 [==============================] - 2s 115us/step - loss: 2.6472
Epoch 4/50
15047/15047 [==============================] - 2s 115us/step - loss: 2.6467
Epoch 5/50
15047/15047 [==============================] - 2s 116us/step - loss: 2.6477
Epoch 6/50
15047/15047 [==============================] - 2s 123us/step - loss: 2.6467
Epoch 7/50
15047/15047 [==============================] - 2s 121us/step - loss: 2.6446
Epoch 8/50
15047/15047 [==============================] - 2s 122us/step - loss: 2.6445
Epoch 9/50
15047/15047 [==============================] - 2s 122us/step - loss: 2.6477
Epoch 10/50
15047/15047 [==============================] - 2s 115us/step - loss: 2.6476
Epoch 11/50
15047/15047 [==============================] - 2s 123us/step - loss: 2.6468
Epoch 12/50
15047/15047 [================

/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as Activation) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


...

Epoch 1/50
15047/15047 [==============================] - 2s 125us/step - loss: 2.5669
Epoch 2/50
15047/15047 [==============================] - 2s 120us/step - loss: 2.5496
Epoch 3/50
15047/15047 [==============================] - 2s 121us/step - loss: 2.5563
Epoch 4/50
15047/15047 [==============================] - 2s 118us/step - loss: 2.5424
Epoch 5/50
15047/15047 [==============================] - 2s 125us/step - loss: 2.5246
Epoch 6/50
15047/15047 [==============================] - 2s 119us/step - loss: 2.5001
Epoch 7/50
15047/15047 [==============================] - 2s 115us/step - loss: 2.5000
Epoch 8/50
15047/15047 [==============================] - 2s 124us/step - loss: 2.6340
Epoch 9/50
15047/15047 [==============================] - 2s 116us/step - loss: 2.6354
Epoch 10/50
15047/15047 [==============================] - 2s 118us/step - loss: 2.6322
Epoch 11/50
15047/15047 [==============================] - 2s 121us/step - loss: 2.6379
Epoch 12/50
15047/15047 [================

/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as Activation) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/50
15047/15047 [==============================] - 2s 123us/step - loss: 2.6412
Epoch 2/50
15047/15047 [==============================] - 2s 117us/step - loss: 2.6279
Epoch 3/50
15047/15047 [==============================] - 2s 123us/step - loss: 2.6290
Epoch 4/50
15047/15047 [==============================] - 2s 114us/step - loss: 2.6309
Epoch 5/50
15047/15047 [==============================] - 2s 117us/step - loss: 2.6332
Epoch 6/50
15047/15047 [==============================] - 2s 117us/step - loss: 2.6327
Epoch 7/50
15047/15047 [==============================] - 2s 124us/step - loss: 2.6327
Epoch 8/50
15047/15047 [==============================] - 2s 114us/step - loss: 2.6324
Epoch 9/50
15047/15047 [==============================] - 2s 122us/step - loss: 2.6327
Epoch 10/50
15047/15047 [==============================] - 2s 125us/step - loss: 2.6323
Epoch 11/50
15047/15047 [==============================] - 2s 115us/step - loss: 2.6327
Epoch 12/50
15047/15047 [================

/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as Activation) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/50
15048/15048 [==============================] - 2s 123us/step - loss: 2.5944
Epoch 2/50
15048/15048 [==============================] - 2s 116us/step - loss: 2.6112
Epoch 3/50
15048/15048 [==============================] - 2s 120us/step - loss: 2.6318
Epoch 4/50
15048/15048 [==============================] - 2s 120us/step - loss: 2.6325
Epoch 5/50
15048/15048 [==============================] - 2s 127us/step - loss: 2.6326
Epoch 6/50
15048/15048 [==============================] - 2s 121us/step - loss: 2.6300
Epoch 7/50
15048/15048 [==============================] - 2s 121us/step - loss: 2.6239
Epoch 8/50
15048/15048 [==============================] - 2s 113us/step - loss: 2.6321
Epoch 9/50
15048/15048 [==============================] - 2s 115us/step - loss: 2.6317
Epoch 10/50
15048/15048 [==============================] - 2s 114us/step - loss: 2.6311
Epoch 11/50
15048/15048 [==============================] - 2s 118us/step - loss: 2.6309
Epoch 12/50
15048/15048 [================

In [0]:
resultados.sum()

-6.657711608698478